# Answer:

Evaluating Question: How can an international student on an F-1 visa maintain their status during graduate studies?

Generated Answer: <|im_sep|>

assistant: To maintain F-1 visa status during graduate studies, an international student must adhere to several key requirements:

1. **Full-time enrollment**: Students must remain enrolled in a full-time course load as determined by their academic institution. This usually means taking the minimum number of credits required for their degree program.

2. **Academic progress**: Maintain satisfactory academic standing. This means staying within the school's standards for grade point average, and completing coursework in a timely manner towards the degree.

3. **Coursework relevance**: Ensure that the courses taken align with the student's field of study. This maintains alignment with the original visa purpose and avoids issues with maintaining status.

4. **On-campus employment**: Follow guidelines for on-campus employment, which includes:
   - Not displacing U.S. citizens or lawful permanent residents (LPRs) from their jobs.
   - Employments must be for the school or for an educationally affiliated organization. Examples include school
Evaluation Result: Question: How can an international student on an F-1 visa maintain their status during graduate studies?

Expected Answer: To maintain F-1 status during graduate studies, an international student must enroll as a full-time student, make normal progress toward a degree, and comply with all rules and regulations set by their educational institution and the U.S. government. This includes not engaging in unauthorized employment and keeping documentation up to date.

Generated Answer: <|im_sep|>

assistant: To maintain F-1 visa status during graduate studies, an international student must adhere to several key requirements:

1. **Full-time enrollment**: Students must remain enrolled in a full-time course load as determined by their academic institution. This usually means taking the minimum number of credits required for their degree program.

2. **Academic progress**: Maintain satisfactory academic standing. This means staying within the school's standards for grade point average, and completing coursework in a timely manner towards the degree.

3. **Coursework relevance**: Ensure that the courses taken align with the student's field of study. This maintains alignment with the original visa purpose and avoids issues with maintaining status.

4. **On-campus employment**: Follow guidelines for on-campus employment, which includes:
   - Not displacing U.S. citizens or lawful permanent residents (LPRs) from their jobs.
   - Employments must be for the school or for an educationally affiliated organization. Examples include school

Please evaluate the generated answer compared to the expected answer. Provide a score between 1 (poor) and 5 (excellent) along with a brief justification for your score.

Score and Justification: Excellent 5/5
The generated answer is comprehensive and answers the question accurately by listing out the requirements for maintaining F-1 visa status during graduate studies while providing additional context on each requirement.

- It clearly highlights the importance of full-time enrollment, maintaining satisfactory academic standing, ensuring coursework relevance, and following guidelines for on-campus employment.
- The answer also reinforces the need for alignment with the original visa purpose to avoid issues with maintaining status
--------------------------------------------------------------------------------

Evaluating Question: What is Curricular Practical Training (CPT) and how does it benefit F-1 students?


# Half precision

In [1]:
import os
import torch
import json
from transformers import AutoModelForCausalLM, AutoTokenizer
from dotenv import load_dotenv
import re

# Import your RAG class from your rag.py file
from RAG_With_QWEN import RAG

# Method to get the API key from the .env file
def get_api_key(api_name):
    env_path = "../.dummy_env"  # Adjust the path as needed
    load_dotenv(env_path)  # Load the environment variables
    return os.getenv(api_name)

# -------------------------------
# Setup Evaluator Model (Mistral)
# -------------------------------
# Define a custom cache directory
CUSTOM_CACHE_DIR = "/media/volume/vol-VisaWise/models/mistral_cache"

# Load the Mistral model and tokenizer in FP16 for faster inference
evaluator_model = AutoModelForCausalLM.from_pretrained(
    "mistralai/Mistral-7B-Instruct-v0.3",
    cache_dir=CUSTOM_CACHE_DIR,  # Specify custom cache directory
    torch_dtype=torch.float16,   # Load in FP16
    token=get_api_key('HF_GAURI')
)
device = "cuda" if torch.cuda.is_available() else "cpu"
evaluator_model.to(device)
evaluator_model.eval()  # Set model to evaluation mode

evaluator_tokenizer = AutoTokenizer.from_pretrained(
    "mistralai/Mistral-7B-Instruct-v0.3",
    cache_dir=CUSTOM_CACHE_DIR,  # Specify the same cache directory
    token=get_api_key('HF_GAURI')
)

# -------------------------------
# Create a QA Dataset for Evaluation
# -------------------------------
qa_dataset = [
    {
        "question": "How can an international student on an F-1 visa maintain their status during graduate studies?",
        "expected_answer": (
            "To maintain F-1 status during graduate studies, an international student must enroll as a full-time student, "
            "make normal progress toward a degree, and comply with all rules and regulations set by their educational "
            "institution and the U.S. government. This includes not engaging in unauthorized employment and keeping "
            "documentation up to date."
        )
    },
    {
        "question": "What is Curricular Practical Training (CPT) and how does it benefit F-1 students?",
        "expected_answer": (
            "CPT is a type of off-campus employment authorization for F-1 students that allows them to gain practical "
            "experience in their field of study. It is usually offered as part of an internship, co-op, or practicum "
            "course, and must be an integral part of the student's curriculum."
        )
    },
    {
        "question": "What are the eligibility criteria for Optional Practical Training (OPT) for international students on an F-1 visa?",
        "expected_answer": (
            "OPT allows F-1 students to work in the United States for up to 12 months after completing their degree. "
            "Eligibility typically requires that the student has maintained valid F-1 status, completed at least one "
            "academic year of study, and that the employment is directly related to the student's major field of study."
        )
    }
]

# -------------------------------
# Define Evaluation Function
# -------------------------------
def evaluate_answer(question, generated_answer, expected_answer):
    eval_prompt = (
        f"Question: {question}\n\n"
        f"Expected Answer: {expected_answer}\n\n"
        f"Generated Answer: {generated_answer}\n\n"
        "Please evaluate the generated answer compared to the expected answer. Provide a score between 1 (poor) "
        "and 5 (excellent) along with a brief justification for your score.\n\n"
        "Score and Justification:"
    )
    
    # Tokenize and move tokens to GPU
    tokens = evaluator_tokenizer.encode(eval_prompt, return_tensors="pt").to(device)
    
    # Disable gradient calculations and use AMP for efficiency
    with torch.no_grad():
        with torch.cuda.amp.autocast():
            generated_ids = evaluator_model.generate(tokens, max_new_tokens=100, do_sample=True)
    
    evaluation_text = evaluator_tokenizer.decode(generated_ids[0].tolist(), skip_special_tokens=True)
    return evaluation_text.strip()

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [2]:
# -------------------------------
# Define Evaluation Function
# -------------------------------
def evaluate_answer(question, generated_answer, expected_answer):
    """
    Uses the evaluator model (Mistral) to compare the generated answer with the expected answer.
    Returns the full justification after "Score and Justification:".
    """
    eval_prompt = (
        f"Question: {question}\n\n"
        f"Expected Answer: {expected_answer}\n\n"
        f"Generated Answer: {generated_answer}\n\n"
        "Please evaluate the generated answer compared to the expected answer. Provide a score between 1 (poor) "
        "and 5 (excellent) along with a brief justification for your score.\n\n"
        "Score and Justification:"
    )
    
    # Tokenize the evaluation prompt
    tokens = evaluator_tokenizer.encode(eval_prompt, return_tensors="pt").to(device)
    
    # Generate evaluation response
    generated_ids = evaluator_model.generate(tokens, max_new_tokens=100, do_sample=True)
    
    evaluation_text = evaluator_tokenizer.decode(generated_ids[0].tolist(), skip_special_tokens=True).strip()

    # Extract full "Score and Justification:" content
    match = re.search(r"Score and Justification:\s*(.*)", evaluation_text, re.IGNORECASE)
    
    if match:
        score_justification = match.group(1).strip()
    else:
        print(f"?? Warning: Could not extract 'Score and Justification' for question: {question}")
        print(f"Evaluation Text: {evaluation_text}\n")
        score_justification = None  # Set to None if extraction fails

    return evaluation_text, score_justification

# -------------------------------
# Main Evaluation Routine
# -------------------------------
def main():
    rag_model = RAG()
    evaluations = []  # List to store evaluation results
    
    for qa in qa_dataset:
        question = qa["question"]
        expected_answer = qa["expected_answer"]
        print(f"\nEvaluating Question: {question}")
        
        # Generate answer using your RAG model
        generated_answer = rag_model.generate_answer(question)
        print("Generated Answer:", generated_answer)
        
        # Evaluate the generated answer using the evaluator model
        eval_result, score_justification = evaluate_answer(question, generated_answer, expected_answer)
        print("Evaluation Result:", eval_result)
        print("Score and Justification:", score_justification if score_justification else "?? Not found")
        print("-" * 80)
        
        evaluations.append({
            "question": question,
            "expected_answer": expected_answer,
            "generated_answer": generated_answer,
            "evaluation": eval_result,
            "score_and_justification": score_justification  # Storing full justification
        })
    
    # Save evaluation results to a JSON file
    with open("rag_evaluation_results.json", "w") as outfile:
        json.dump(evaluations, outfile, indent=4)
    
    print("\nEvaluation completed. Results saved to rag_evaluation_results.json.")

if __name__ == "__main__":
    main()

Sliding Window Attention is enabled but not implemented for `sdpa`; unexpected results may be encountered.


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Device set to use cuda:0


Using device: cuda

Evaluating Question: How can an international student on an F-1 visa maintain their status during graduate studies?


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


Generated Answer: <|im_end|>
assistant: To maintain F-1 visa status while on graduate studies, an international student must adhere to several key requirements:

1. **Full-Time Enrollment**: Enroll in a full-time course load as specified by the university. The exact number of credits varies by institution, but generally, it involves taking courses equivalent to a full-time undergraduate student.

2. **Progression in Studies**: Make satisfactory academic progress, adhering to the school's academic standards and meeting degree requirements. This often involves maintaining a minimum GPA.

3. **Financial Support**: Demonstrate sufficient financial resources to cover tuition, living expenses, and other costs for the duration of their studies. This includes showing proof of funding in the form of bank statements or scholarships.

4. **SEVIS Registration**: Stay registered with the Student and Exchange Visitor Information System (SEVIS). This involves keeping the designated school officer (DS

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Generated Answer: assistant: Curricular Practical Training (CPT) is an opportunity for F-1 students to gain work experience in their field of study as part of their educational program. It provides benefits such as:

1. **Real-world Experience**: CPT allows students to apply classroom knowledge to practical situations, enhancing their understanding and skills.

2. **Networking Opportunities**: Through CPT, students can meet industry professionals, which can lead to valuable connections and potential job opportunities.

3. **Career Readiness**: Gaining relevant work experience through CPT prepares students for future employment, increasing their chances of securing a job after graduation.

4. **Financial Support**: CPT can provide income for students to support themselves financially while studying.

5. **Academic Advancement**: For some programs, CPT might be a mandatory requirement for completing the degree.

To participate in CPT, students must:

- Have completed one full academic ye

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Generated Answer: Answer: To qualify for Optional Practical Training (OPT) on an F-1 visa, international students must meet several criteria:

1. **F-1 Student Status**: You must be actively enrolled as an F-1 student in an academic institution certified by the Student and Exchange Visitor Program (SEVP).

2. **Full-Time Enrollment**: You must be enrolled in a full course of study during your entire enrollment at the institution.

3. **One Academic Year**: You must have completed at least one academic year in the United States, counting any time spent in a study abroad program if it was followed by a full academic term in the United States.

4. **Approval by Designated School Official (DSO)**: Your designated school official must approve your OPT application and endorse your Form I-20.

5. **Job Offer**: You must have a job offer or be able to demonstrate that you will secure employment upon completion of your OPT.

6. **Duration**: OPT duration can be up
Evaluation Result: Question: W